## [Ensembles](https://en.wikipedia.org/wiki/Ensemble_learning)
**Descrição**: São _meta-algoritmos_ que combinam várias técnicas de aprendizado de máquina em um único modelo preditivo ([Wisdom of the crowd](https://en.wikipedia.org/wiki/Wisdom_of_the_crowd)).<br>
**Objetivo**: Prover estabilidade. Diminuir a variância (bagging), viés (boosting) ou melhorar as predições (stacking).<br>
**Estratégia**: Combinar as predições de modelos especializados. Um grupo de modelos _fracos_ juntos formam um modelo _forte_ ([United we stand](https://en.wikipedia.org/wiki/United_we_stand,_divided_we_fall)).<br>
**Problemas**: Podem ser aplicados tanto em problemas de classificação quanto em regressão.<br>
**Estruturas**:
- Paralelo: Os modelos (heterogêneos ou homogêneos) são treinados paralelamente de forma indepentende e então combinados para gerar a previsão.
- Sequencial: Os modelos são treinados sequencialmente de modo que o erro seja minimizado ao longo da sequência.<br>

**Premissas** por *James Michael Surowiecki sobre Wisdom of the crowd*:
- Independência: Capacidade de formar uma opinião independente da opinião alheia.
- Decentralização: Capacidade de especializar e tirar conclusões baseado em informação local.
- Diversidade: Capacidade de manter informação privada mesmo que seja uma opinião enviesada.
- Agregação: Algum mecanismo capaz de tornar julgamentos privados em uma decisão coletiva.<br>

**Observações**:
- Não são recomendados quando é importante ter interpretabilidade e explicabilidade
- Tempo de computação e desenvolvimento é alto!
- Selecionar modelos para criar ensembles não é trivial
- Nem sempre irá produzir resultados bons

## Bias-Variance Tradeoff

<img src="bias-variance-tradeoff.svg" width ="300" height=300 />

$\text{Erro} = \text{Viés}(\hat\theta, \theta) + \text{Variância}(\hat\theta, \theta) + \text{Ruído}$

## Modelo Baseline

In [2]:
from mlxtend.evaluate import bias_variance_decomp
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification

<img src="simple.svg" width ="500" height=500 />

In [3]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

In [4]:
tree = DecisionTreeClassifier(max_depth=3, random_state=1)

# A métrica 0-1_loss é calculada da seguinte forma:
# se y != ŷ então 1 senão 0
# O ruido é ignorado por uma questão de simplificação
loss, bias, var = bias_variance_decomp(tree, X_train, y_train, X_test, y_test, loss='0-1_loss', random_seed=0)

print('%.3f Average expected loss' % loss)
print('%.3f Average bias' % bias)
print('%.3f Average variance' % var)

0.223 Average expected loss
0.160 Average bias
0.157 Average variance


## [Boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)
- Estrutura sequencial
- Objetivo é reduzir o viés (bias) do modelo (não a variância)
- Adequado para modelos com baixa variância e alto viés (Baseados em Árvores)
- Baseado na [Hypothesis Boosting](https://en.wikipedia.org/wiki/Boosting_(machine_learning)) a qual assume que um conjunto de modelos fracos são capazes de produzir modelos fortes.
- Exemplos comuns: [ExtraTrees](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier), [GradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier), [HistGradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html#sklearn.ensemble.HistGradientBoostingClassifier)

<img src="boosting.svg" width ="500" height=500 />

In [5]:
tree = DecisionTreeClassifier(max_depth=3, random_state=1)
boosting = AdaBoostClassifier(base_estimator=tree, n_estimators=15, random_state=1)

loss, bias, var = bias_variance_decomp(boosting, X_train, y_train, X_test, y_test, loss='0-1_loss', random_seed=1)

print('%.3f Average expected loss' % loss)
print('%.3f Average bias' % bias)
print('%.3f Average variance' % var)

0.150 Average expected loss
0.068 Average bias
0.131 Average variance


## [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier) (aka Bootstrap AGGregatING)
- Estrutura paralela e faz a combinação dos modelos usando votação (classificação) ou média (regressão)
- Objetivo é reduzir a variância do modelo (não o viés)
- Adequado para modelos com alta variância e baixo viés (modelos complexos) e conjuntos de dados pequenos/moderados em número de instâncias.
- Baseado na estratégia de [Bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) a qual faz uma _amostragem com reposição_ de mesma cardinalidade do conjunto de dados original permitindo assim reduzir a variância dos dados.
- Exemplo comum: [RandomForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

<img src="bagging.svg" width ="700" height=700 />

In [5]:
tree = DecisionTreeClassifier(max_depth=3, random_state=1)
bagging = BaggingClassifier(base_estimator=tree, n_estimators=15, random_state=1)

loss, bias, var = bias_variance_decomp(bagging, X_train, y_train, X_test, y_test, loss='0-1_loss', random_seed=1)

print('%.3f Average expected loss' % loss)
print('%.3f Average bias' % bias)
print('%.3f Average variance' % var)

0.168 Average expected loss
0.140 Average bias
0.096 Average variance


## [Stacking](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier) (aka Stacked Generalization)
- Estrutura paralela e faz combinação dos modelos usando um modelo (_e.g._ [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html))
- Objetivo é aumentar o poder preditivo (diminuir o erro) do modelo
- Baseado na ideia de que diferentes modelos podem aprender diferentes padrões nos dados mas não o suficiente para generalizar a maioria dos padrões. Dessa forma, a estratégia é usar um modelo que generalize os padrões encontrados por diferentes modelos.


<img src="stacking.svg" width ="600" height=600 />

In [6]:
estimators = [
    ('rdg', RidgeClassifier(normalize=True)),
    ('gnb', GaussianNB()),
    ('knn', KNeighborsClassifier(n_neighbors=3)),
    ('dt' , DecisionTreeClassifier(max_depth=10, random_state=1)),
    ('svm', make_pipeline(StandardScaler(), SVC(random_state=42)))
]

stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()
)

loss, bias, var = bias_variance_decomp(stacking, X_train, y_train, X_test, y_test, loss='0-1_loss', random_seed=1)

print('%.3f Average expected loss' % loss)
print('%.3f Average bias' % bias)
print('%.3f Average variance' % var)

0.048 Average expected loss
0.040 Average bias
0.019 Average variance


## Referências

1. [Bagging Ensemble with Python](https://machinelearningmastery.com/bagging-ensemble-with-python/)
2. [Stacking Ensemble with Python](https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/)
3. [Ensemble Learning](https://blog.statsbot.co/ensemble-learning-d1dcd548e936)
4. [Main Approaches for Ensemble Learning](https://www.kdnuggets.com/2019/01/ensemble-learning-5-main-approaches.html)
5. [Simple Guide for Ensemble Leraning Methods](https://towardsdatascience.com/simple-guide-for-ensemble-learning-methods-d87cc68705a2)
6. [Bias-Variance Decomposition](http://rasbt.github.io/mlxtend/user_guide/evaluate/bias_variance_decomp/)
7. [MIT - Boosting](https://www.youtube.com/watch?v=UHBmv7qCey4)
8. [Stacking and Blending](https://medium.com/@stevenyu530_73989/stacking-and-blending-intuitive-explanation-of-advanced-ensemble-methods-46b295da413c)
9. [Imagem: Bias-Variance Tradeoff](https://www.machinelearningtutorial.net/wp-content/uploads/2017/01/bias-variance-tradeoff.svg)

## Bibliotecas

1. [vecstack](https://github.com/vecxoz/vecstack)
2. [mlens](https://github.com/flennerhag/mlens)